In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

from scipy.stats import norm

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)

import sys
sys.path.insert(0,'../cmds')
from options import *

import nasdaqdatalink as ndl

***

In [2]:
DATE0 = '2024-09-01'
DATE1 = '2025-01-31'

ticks = [
    'ES','T','NG','GC','BP','TY','FF'
]
codes_exch = [
    'CME','ICE','NYM','CMX','CME','CBT','CBT'
]
names = [
    'S&P','Oil','Nat Gas','Gold','British Pounds','Treasury Notes','Fed Funds'
]
expmonth = 'M2025'

In [3]:
file_key = open("../../keys/quandl_key.txt","r")
API_KEY = file_key.read()
file_key.close()
ndl.ApiConfig.api_key = API_KEY

In [4]:
surfs = dict()
labels = dict()

ticksplitlabs = ['name','exchange','futures ticker','option ticker','monthYear']
info = pd.DataFrame(index=ticksplitlabs)
info.index.name = 'specs'

In [5]:
for i in range(len(ticks)):

    tick = ticks[i]
    exch = codes_exch[i]
    
    tempM = ndl.get_table('AR/IVM',exchange_code=exch, futures_code=tick, option_code=tick, date={'gte':DATE0},expiration=expmonth).set_index('date').sort_index()
    tempS = ndl.get_table('AR/IVS',exchange_code=exch, futures_code=tick, option_code=tick, date={'gte':DATE0},expiration=expmonth).set_index('date').sort_index()

    id_cols = ['futures','days_expiration','days_termination']
    pattern_columns = [col for col in tempS.columns if (col.startswith(('p', 'c')) and col.endswith('vol'))]

    aa = tempM[id_cols]
    bb = tempS[pattern_columns]
    bb.columns = bb.columns.str.capitalize()

    out = pd.concat([aa,bb],axis=1)
    out[['days_expiration', 'days_termination']] = out[['days_expiration', 'days_termination']] / 365
    out.rename(columns={'futures':'Future Price', 'days_termination':'Expiration Future', 'days_expiration':'Expiration Option'},inplace=True)

    # Remove estimates deeply ITM/OTM
    substrings = ['01', '05', '10','90','95','99']
    out = out[[col for col in out.columns if not any(sub in col for sub in substrings)]]

    surfs[tick] = out

    label = f'{exch} {tick} {tick} {expmonth}'    
    labels[tick] = label

    if len(out.index)>0:
        finalDate = out.index[-1]
        #finalDate = DATE1

        info.loc['name',label] = names[i]
        info.loc['exchange',label] = exch
        info.loc['futures ticker',label] = tick
        info.loc['option ticker',label] = tick
        info.loc['monthYear',label] = expmonth
        
        info.loc['option expiration',label] = (finalDate + datetime.timedelta(tempM.loc[finalDate,'days_expiration'])).strftime('%Y-%m-%d')
        info.loc['future expiration',label] = (finalDate + datetime.timedelta(tempM.loc[finalDate,'days_termination'])).strftime('%Y-%m-%d')


In [6]:
with pd.ExcelWriter(f'../data/vol_surfaces_{DATE1}_{expmonth}.xlsx') as writer:  
    info.to_excel(writer, sheet_name= 'descriptions')
    for tick in ticks:
        surfs[tick].to_excel(writer, sheet_name=labels[tick])